# Assignment: Clustering and association rule mining

## 1. Business Understanding

The company has a fleet of drones that deliver products to customers.
The company intends to create a set of hubs, or depots, where the drones are stationed, serviced, and loaded with products for delivery.
The company is looking to optimize its operations by finding the optimal locations for its hubs.

Another optimization idea is to increase the revenue of the company by recommending products to customers based on their purchase history. For this purpose, the company is looking to use association rule mining to find interesting relationships between products that are frequently purchased together.
In the initial phase of the project, the company focuses on 20 internal product groups, and wants to find out which product groups act as triggers for buying items from some other product groups.

## Task

The task is split into two sub-tasks: finding optimal hub locations and finding interesting relationships between product groups.